In [ ]:
import cv2
import os
import skimage
from skimage import data
from skimage import transform
import PIL
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random
from random import randint
import tensorflow as tf

In [ ]:
def load_data(data_directory):
    directories = [d for d in os.listdir(data_directory) 
                   if os.path.isdir(os.path.join(data_directory, d))]
    labels = []
    images = []
    for d in directories:
        label_directory = os.path.join(data_directory, d)
        file_names = [os.path.join(label_directory, f) 
                      for f in os.listdir(label_directory) if f.endswith(".jpg")]
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(str(d))
    return images, labels

In [ ]:
ROOT_PATH = '/host//Can/dpr/'
train_data_directory = os.path.join(ROOT_PATH, "outPut/train")
#test_data_directory = os.path.join(ROOT_PATH, "outPut/val")

In [ ]:
Images,Labels= load_data(train_data_directory)

In [ ]:
plt.hist(Labels,36)

In [ ]:
imgplot = plt.imshow(Images[0])


In [ ]:

Labels

In [ ]:
#add value 15 in hsv images,yellow will change into green
i=0
for p in range(0,32*1):
    
    img = Images[i]
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    turn_green_hsv = img_hsv.copy()
    turn_green_hsv[:, :, 0] = (turn_green_hsv[:, :, 0]+30) % 180
    turn_green_img = cv2.cvtColor(turn_green_hsv, cv2.COLOR_HSV2BGR)
    filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+'_hsv_30h'+'.jpg'
    plt.imsave(filename, turn_green_img)
    print(filename)
    i+=1


In [ ]:
#saturation half of before
i=0
for p in range(0,32):
    
    img = Images[i]
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    colorless_hsv = img_hsv.copy()
    colorless_hsv[:, :, 1] = 0.2 * colorless_hsv[:, :, 1]
    colorless_img = cv2.cvtColor(colorless_hsv, cv2.COLOR_HSV2BGR)
    

    filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+'_hsv_02s'+'.jpg'
    plt.imsave(filename, colorless_img)
    print(filename)
    i+=1


In [ ]:
plt.imshow(Images[1])

In [ ]:
#value change into half before
i=0

for p in range(0,32):
    img = Images[i]
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    darker_hsv = img_hsv.copy()
    darker_hsv[:, :, 2] = 0.1 * darker_hsv[:, :, 2]
    darker_img = cv2.cvtColor(darker_hsv, cv2.COLOR_HSV2BGR)
    
    filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+'_hsv_01v'+'.jpg'
    plt.imsave(filename, darker_img)
    print(filename)
    i+=1
        
    

In [ ]:
#gamma_trans
def gamma_trans(img, gamma):
    # 具体做法是先归一化到1，然后gamma作为指数值求出新的像素值再还原
    gamma_table = [np.power(x/255.0, gamma)*255.0 for x in range(256)]
    gamma_table = np.round(np.array(gamma_table)).astype(np.uint8)
    
    # 实现这个映射用的是OpenCV的查表函数
    return cv2.LUT(img, gamma_table)
#chang gamma into half of before
i=0
for p in range(0,32):
    img = Images[i]
    for j in range(1,9):#8 images with gamma
        img_corrected = gamma_trans(img,0.1*j)
    
        filename=train_data_directory+'/'+Labels[i]+'/'+ Labels[i]+'_gamma0'+str(j)+'.jpg'
        plt.imsave(filename, img_corrected)
    print(filename)
    i+=1
        
    




In [ ]:
#######################################################################################################3
########################################################################################################
##########################################################################################################
#projective transformer


In [ ]:
#vertically enlarge 1.6times, trans -2,-1 and crop elephant:)
M_crop_elephant = np.array([
    [1.6, 0, -2],
    [0, 1.6, -1]
], dtype=np.float32)

img_cr = cv2.warpAffine(testImages[0],M_crop_elephant,(8,16))

In [ ]:
plt.imshow(img_cr)

In [ ]:
i=0
M_crop_elephant = np.array([
    [1.6, 0, -2],
    [0, 1.6, -1]
], dtype=np.float32)
for p in range(0,32):
    img = Images[i]
    for j in range(1,9):#8 images with gamma
        M_crop_elephant = np.array([[1.0+0.1*j, 0, -2],[0, 1.0+0.1*j, -1]], dtype=np.float32)
       
        img_elephant = cv2.warpAffine(img, M_crop_elephant, (8,16))
    
        filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+str(j)+'_vertical1' +'_translation2-1'+'.jpg'
        plt.imsave(filename, img_elephant)
    print(filename)
    i+=1

In [ ]:
#shear transform
#shear angle 15 degree
i=0
theta = 15 * np.pi / 180
M_shear = np.array([
    [1, np.tan(theta), 0],
    [0, 1, 0]
], dtype=np.float32)
img_sheared = cv2.warpAffine(testImages[0], M_shear, (8, 16))
plt.imshow(img_sheared)


In [ ]:
theta = 15 * np.pi / 180
M_shear = np.array([
    [1, np.tan(theta), 0],
    [0, 1, 0]
], dtype=np.float32)
i=0
for p in range(0,32):
    img = Images[i]
    for j in range(0,5):
        theta = 15 *j* np.pi / 180
        M_shear = np.array([[1, np.tan(theta), 0],[0, 1, 0]], dtype=np.float32)
        
    
        img_sheared = cv2.warpAffine(img, M_shear, (8,16))
    
        filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+'_sheared'+str(15*j)+'.jpg'
        plt.imsave(filename, img_sheared)
    print(filename)
    i+=1

In [ ]:
#rotation rotated 15 degree
i=0
M_rotate = np.array([
    [np.cos(theta), -np.sin(theta), 0],
    [np.sin(theta), np.cos(theta), 0]
], dtype=np.float32)
i=0
for p in range(0,32):
    img = Images[i]
    
    for j in range(1,5):
        theta = 15 *j* np.pi / 180
        M_rotate = np.array([[np.cos(theta), -np.sin(theta), 0],[np.sin(theta), np.cos(theta), 0]], dtype=np.float32)
        img_rotated15 = cv2.warpAffine(img, M_rotate, (8,16))
    
        filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+'_rotate'+str(15*j)+'.jpg'
        plt.imsave(filename, img_rotated15)
    print(filename)
    i+=1

In [ ]:
#general projective transform
M = np.array([
    [0.1, 0.5, -2],
    [0.5, 0.1, -1]
], dtype=np.float32)

i=0
for p in range(0,32):
    img = Images[i]
    for j in range(0,100):
        M = np.array([[random.uniform(1,2), random.uniform(0,1), random.uniform(-1,1) ],
                      [random.uniform(0,1), random.uniform(1,2), random.uniform(-1,1) ]], dtype=np.float32)
        img_projective = cv2.warpAffine(img, M, (8,16))
    
        filename=train_data_directory+'/'+Labels[i]+'/'+Labels[i]+'_projective_random'+str(int(j))+'.jpg'
        plt.imsave(filename, img_projective)
    print(filename)
    i+=1